# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import re
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

# magic word for producing visualizations in notebook
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# load data from database
def load_data():
    engine = create_engine('sqlite:///DisasterResponse.db')
    df =  pd.read_sql_table('DisasterResponse', engine)
    X = df.message.values
    y = df.iloc[:,5:]
    return X, y

In [3]:
X, y = load_data()

### 2. Write a tokenization function to process your text data

In [4]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [5]:
def tokenize(text):
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [9]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    accuracy = (y_pred == y_test).mean()
    print("Labels:", labels)
    print("Accuracy:", accuracy)

In [11]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [12]:
# display results
display_results(y_test, y_pred)

Labels: [0 1]
Accuracy: request                   0.881119
offer                     0.995423
aid_related               0.728417
medical_help              0.921170
medical_products          0.949396
search_and_rescue         0.975842
security                  0.981310
military                  0.969231
child_alone               1.000000
water                     0.948379
food                      0.912905
shelter                   0.923967
clothing                  0.985633
money                     0.979148
missing_people            0.988811
refugees                  0.966688
death                     0.959822
other_aid                 0.867260
infrastructure_related    0.935664
transport                 0.954609
buildings                 0.951939
electricity               0.979530
tools                     0.993897
hospitals                 0.990083
shops                     0.996440
aid_centers               0.986904
other_infrastructure      0.956516
weather_related           0.842

### 6. Improve your model
Use grid search to find better parameters. 

In [31]:
parameters = {
    'estimator__bootstrap': [True, False],
    'estimator__class_weight': [None, 'balanced', 'balanced_subsample'],
    'estimator__criterion': ['gini', 'entropy'],
    'estimator__max_depth': [None, 10, 20, 30, 50],
    'estimator__max_features': ['auto', None, 'sqrt', 'log2'],
    'estimator__max_leaf_nodes': [None, 10, 25, 50, 100],
    'estimator__min_impurity_decrease': [0.00],
    'estimator__min_impurity_split': [None],
    'estimator__min_samples_leaf': [1, 2, 10],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_weight_fraction_leaf':[0.0],
    'estimator__n_estimators': [10,20, 40, 50],
    'estimator__n_jobs':[1],
    'estimator__oob_score': [False], 
    'estimator__random_state': [None], 
    'estimator__verbose': [0],
    'estimator__warm_start': [False],
    'estimator': ['MultiOutputClassifier'],
    'n_jobs': [1]
}

cv = GridSearchCV(estimator=clf, param_grid=parameters)

In [32]:
clf.get_params().keys()

dict_keys(['estimator__bootstrap', 'estimator__class_weight', 'estimator__criterion', 'estimator__max_depth', 'estimator__max_features', 'estimator__max_leaf_nodes', 'estimator__min_impurity_decrease', 'estimator__min_impurity_split', 'estimator__min_samples_leaf', 'estimator__min_samples_split', 'estimator__min_weight_fraction_leaf', 'estimator__n_estimators', 'estimator__n_jobs', 'estimator__oob_score', 'estimator__random_state', 'estimator__verbose', 'estimator__warm_start', 'estimator', 'n_jobs'])

In [33]:
clf.get_params().values()

dict_values([True, None, 'gini', None, 'auto', None, 0.0, None, 1, 2, 0.0, 10, 1, False, None, 0, False, RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 1])

In [34]:
cv.fit(X_train,y_train)    
#best_score = grid_search.best_score_
#best_params = grid_search.best_params_

ValueError: The base estimator should implement a fit method

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.